<a href="https://colab.research.google.com/github/terkoizmy/TweetDisaster/blob/main/Tweet_Disaster_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Library

In [162]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
import xgboost as xgb
from xgboost import XGBClassifier
from tensorflow.python.keras.preprocessing.text import Tokenizer
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [164]:
df_train = pd.read_csv('train.csv')

In [165]:
df_train.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


# Data processing text

In [166]:
wl = WordNetLemmatizer()
def preprocess_text(texts):
  corpus = []
  for text in texts:
    text = re.sub(r'https?://\S+|www\.\S+','',text)
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    text = text.split()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    text = [wl.lemmatize(word) for word in text if not word in set(all_stopwords)]
    text = ' '.join(text)
    corpus.append(text)

  return corpus

In [167]:
df_train['processed_text'] = preprocess_text(df_train['text'])

In [168]:
print(df_train['text'][0:5])
print(df_train['processed_text'][0:5])

0    Our Deeds are the Reason of this #earthquake M...
1               Forest fire near La Ronge Sask. Canada
2    All residents asked to 'shelter in place' are ...
3    13,000 people receive #wildfires evacuation or...
4    Just got sent this photo from Ruby #Alaska as ...
Name: text, dtype: object
0           deed reason earthquake may allah forgive u
1                forest fire near la ronge sask canada
2    resident asked shelter place notified officer ...
3    people receive wildfire evacuation order calif...
4    got sent photo ruby alaska smoke wildfire pour...
Name: processed_text, dtype: object


# Bag of Word

In [169]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1600)
X = cv.fit_transform(df_train['processed_text']).toarray()
y = df_train['target'].values

In [170]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25)

# Creat model

## Naive Bayes

In [171]:
from sklearn.naive_bayes import GaussianNB
Naive = GaussianNB()
Naive.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [172]:
NBpred = Naive.predict(X_test)

In [173]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, NBpred)
print(cm)
accuracy_score(y_test, NBpred)

[[975 107]
 [319 503]]


0.7762605042016807

## Random Forest

In [174]:
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
# random forest model creation
rfc = RandomForestClassifier()
rfc.fit(X_train,y_train)
# predictions
rfc_pred = rfc.predict(X_test)

In [175]:
cm = confusion_matrix(y_test, rfc_pred)
print(cm)
accuracy_score(y_test, rfc_pred)

[[896 186]
 [240 582]]


0.7762605042016807

# 4Submission

## Train full data

Random Forest

In [176]:
rfc_full = RandomForestClassifier()
rfc_full.fit(X,y)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

Naive Bayes

In [177]:
Naive_full = GaussianNB()
Naive_full.fit(X, y)

GaussianNB(priors=None, var_smoothing=1e-09)

In [178]:
df_test = pd.read_csv('test.csv')
df_test['processed_text'] = preprocess_text(df_test['text'])

In [179]:
print(df_test['text'][0:5])
print(df_test['processed_text'][0:5])

0                   Just happened a terrible car crash
1    Heard about #earthquake is different cities, s...
2    there is a forest fire at spot pond, geese are...
3             Apocalypse lighting. #Spokane #wildfires
4        Typhoon Soudelor kills 28 in China and Taiwan
Name: text, dtype: object
0                          happened terrible car crash
1    heard earthquake different city stay safe ever...
2    forest fire spot pond goose fleeing across str...
3                 apocalypse lighting spokane wildfire
4                   typhoon soudelor kill china taiwan
Name: processed_text, dtype: object


In [180]:
test_data = cv.fit_transform(df_test['processed_text']).toarray()

In [181]:
NBpredict = Naive_full.predict(test_data)
RFpredict = rfc_full.predict(test_data)

In [182]:
submit_NB = pd.DataFrame()
submit_NB['id'] = df_test['id']
submit_NB['target'] = NBpredict



submit_RF = pd.DataFrame()
submit_RF['id'] = df_test['id']
submit_RF['target'] = RFpredict

In [183]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [184]:
submit_NB.to_csv('/content/gdrive/MyDrive/Submission/submit_NB.csv',index=False)

submit_RF.to_csv('/content/gdrive/MyDrive/Submission/submit_RF.csv',index=False)